# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [31]:
#import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger','punkt_tab','averaged_perceptron_tagger_eng'])

import re
import pickle
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer




[nltk_data] Downloading package punkt to /Users/macos/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/macos/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/macos/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt_tab to /Users/macos/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/macos/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


In [4]:
# load data from database
engine = create_engine('sqlite:///../data/disaster_response.db')
df = pd.read_sql_table('disaster_response', con=engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
pd.set_option('display.max_columns', None)


In [4]:
df.describe()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26216.00000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.0,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000
mean,15224.82133,0.773650,0.170659,0.004501,0.414251,0.079493,0.050084,0.027617,0.017966,0.032804,0.0,0.063778,0.111497,0.088267,0.015449,0.023039,0.011367,0.033377,0.045545,0.131446,0.065037,0.045812,0.050847,0.020293,0.006065,0.010795,0.004577,0.011787,0.043904,0.278341,0.082202,0.093187,0.010757,0.093645,0.020217,0.052487,0.193584
std,8826.88914,0.435276,0.376218,0.066940,0.492602,0.270513,0.218122,0.163875,0.132831,0.178128,0.0,0.244361,0.314752,0.283688,0.123331,0.150031,0.106011,0.179621,0.208500,0.337894,0.246595,0.209081,0.219689,0.141003,0.077643,0.103338,0.067502,0.107927,0.204887,0.448191,0.274677,0.290700,0.103158,0.291340,0.140743,0.223011,0.395114
min,2.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7446.75000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15662.50000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22924.25000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30265.00000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [5]:
df.iloc[:,4:].mean()

related                   0.773650
request                   0.170659
offer                     0.004501
aid_related               0.414251
medical_help              0.079493
medical_products          0.050084
search_and_rescue         0.027617
security                  0.017966
military                  0.032804
child_alone               0.000000
water                     0.063778
food                      0.111497
shelter                   0.088267
clothing                  0.015449
money                     0.023039
missing_people            0.011367
refugees                  0.033377
death                     0.045545
other_aid                 0.131446
infrastructure_related    0.065037
transport                 0.045812
buildings                 0.050847
electricity               0.020293
tools                     0.006065
hospitals                 0.010795
shops                     0.004577
aid_centers               0.011787
other_infrastructure      0.043904
weather_related     

- We remove *child_alone* as it only contains 0 values 


In [6]:
df = df.drop('child_alone', axis = 1)


In [7]:
df.isnull().sum()

id                            0
message                       0
original                  16046
genre                         0
related                       0
request                       0
offer                         0
aid_related                   0
medical_help                  0
medical_products              0
search_and_rescue             0
security                      0
military                      0
water                         0
food                          0
shelter                       0
clothing                      0
money                         0
missing_people                0
refugees                      0
death                         0
other_aid                     0
infrastructure_related        0
transport                     0
buildings                     0
electricity                   0
tools                         0
hospitals                     0
shops                         0
aid_centers                   0
other_infrastructure          0
weather_

In [8]:
df[['id','related']].groupby('related').count()

,id
related,
0,6122
1,19906
2,188


- *related* = 2 should be a bug, we need to replace them by 0 or 1. In this case, I will go with 1


In [9]:
df['related'] = df['related'].map(lambda x: 1 if x == 2 else x)

In [10]:
df[['id','related']].groupby('related').count()

,id
related,
0,6122
1,20094


In [11]:
df.sample()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
24190,27969,"As an immediate support, women and men from 50...",None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
X = df['message']
y = df.iloc[:,4:]

In [13]:
pd.set_option('display.max_colwidth', None)

In [14]:
df['message'].sample(10)

22544                                                                                                                                                 Includes cassava maize equivalent; Source: National and Regional EWUs
9361                                                 Great I am nesly please tell me whether it is true that a volcano in Saut-d'Eau, and I want to know whether the election is effectively true, if yes when he is doing 
8106                                                                                                                        I need a card to find some thing to eat, I hope that, may God bless you. School going to open. 
11187                                                                                                                    Local fame has been achieved .. . HAHAHA the local representative of hurricane sandy right here ..
6902                                                                                                                    

### 2. Write a tokenization function to process your text data

In [15]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [16]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import confusion_matrix, classification_report



In [17]:
clf = RandomForestClassifier()

In [18]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(clf, n_jobs= -1))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

model = pipeline
model.fit(X_train, y_train)
y_pred = model.predict(X_test)


/Users/macos/Library/Python/3.9/lib/python/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [23]:
# classification report on test data
print(classification_report(y_test.values, y_pred, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.81      0.97      0.88      5008
               request       0.86      0.44      0.58      1098
                 offer       0.00      0.00      0.00        27
           aid_related       0.80      0.63      0.70      2712
          medical_help       0.73      0.04      0.08       532
      medical_products       0.81      0.04      0.07       346
     search_and_rescue       1.00      0.05      0.09       184
              security       0.00      0.00      0.00       126
              military       0.71      0.02      0.04       233
                 water       0.89      0.23      0.37       412
                  food       0.86      0.36      0.50       707
               shelter       0.84      0.24      0.38       590
              clothing       0.60      0.03      0.05       112
                 money       0.67      0.03      0.05       151
        missing_people       0.00      

/Users/macos/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/macos/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/macos/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users

### 6. Improve your model
Use grid search to find better parameters. 

In [24]:
# get list of model parameters
model.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x12b2ae1f0>)),
  ('tfidf', TfidfTransformer()),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(), n_jobs=-1))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x12b2ae1f0>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier(), n_jobs=-1),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_t

- Using GridSearchCV to train the model takes a lot of time, especially as more parameters are added. For demonstration purposes, only a small set of parameters is used. To get a more optimized model, we would need to test more parameters, which would take more time and resources.

In [35]:


parameters1 = {
    'clf__estimator__max_depth': [100, 200, 300, 500],
    'clf__estimator__n_estimators': [None, 10, 20, 30, 40]
}


cv = GridSearchCV(model, param_grid=parameters1, cv=5, n_jobs=-1, verbose=3) 
# verbose=3 to get real time training progress
# n_jobs=-1 -> to train in parallel across the maximum number of cores in our computer, spending it up.
# cv=5 -> 5-fold cross validation


cv.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


/Users/macos/Library/Python/3.9/lib/python/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/macos/Library/Python/3.9/lib/python/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 1/5] END clf__estimator__max_depth=100, clf__estimator__n_estimators=None;, score=nan total time=  13.2s


/Users/macos/Library/Python/3.9/lib/python/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 2/5] END clf__estimator__max_depth=100, clf__estimator__n_estimators=None;, score=nan total time=  11.5s


...

[CV 3/5] END clf__estimator__max_depth=500, clf__estimator__n_estimators=30;, score=0.249 total time= 3.5min


/Users/macos/Library/Python/3.9/lib/python/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 4/5] END clf__estimator__max_depth=500, clf__estimator__n_estimators=30;, score=0.235 total time= 3.6min
[CV 5/5] END clf__estimator__max_depth=500, clf__estimator__n_estimators=30;, score=0.238 total time= 3.8min
[CV 1/5] END clf__estimator__max_depth=500, clf__estimator__n_estimators=40;, score=0.242 total time= 4.6min
[CV 2/5] END clf__estimator__max_depth=500, clf__estimator__n_estimators=40;, score=0.241 total time= 4.3min
[CV 3/5] END clf__estimator__max_depth=500, clf__estimator__n_estimators=40;, score=0.241 total time= 3.9min
[CV 4/5] END clf__estimator__max_depth=500, clf__estimator__n_estimators=40;, score=0.239 total time= 3.4min
[CV 5/5] END clf__estimator__max_depth=500, clf__estimator__n_estimators=40;, score=0.239 total time= 3.0min


/Users/macos/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
20 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/Users/macos/Library/Python/3.9/lib/python/site-packages/joblib/_utils.py", line 72, in __call__
    return self.func(**kwargs)
  File "/Users/macos/Library/Python/3.9/lib/python/site-packages/joblib/parallel.py", line 598, in __call__
    return [func(*args, **kwargs)
  File "/Users/macos/Library/Python/3.9/lib/python/site-packages/joblib/parallel.py", line 598, in <listcomp>
    return [fun

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x12b2ae1f0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(),
                                                              n_jobs=-1))]),
             n_jobs=-1,
             param_grid={'clf__estimator__max_depth': [100, 200, 300, 500],
                         'clf__estimator__n_estimators': [None, 10, 20, 30,
                                                          40]},
             verbose=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [36]:
y_pred_test = cv.predict(X_test)

# classification report on test data
print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.81      0.97      0.89      5008
               request       0.87      0.43      0.58      1098
                 offer       0.00      0.00      0.00        27
           aid_related       0.78      0.60      0.68      2712
          medical_help       0.79      0.07      0.13       532
      medical_products       0.90      0.05      0.10       346
     search_and_rescue       0.78      0.04      0.07       184
              security       0.00      0.00      0.00       126
              military       0.75      0.04      0.07       233
                 water       0.89      0.19      0.32       412
                  food       0.85      0.46      0.60       707
               shelter       0.84      0.28      0.42       590
              clothing       0.87      0.12      0.20       112
                 money       0.67      0.03      0.05       151
        missing_people       0.50      

/Users/macos/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/macos/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/macos/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users

In [39]:
cv.best_params_

{'clf__estimator__max_depth': 500, 'clf__estimator__n_estimators': 40}

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [27]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [33]:
# include StartingVerbExtractor custom transformer and try AdaBoostClassifier()
clf2 = AdaBoostClassifier()

model_2 = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
        ('starting_verb', StartingVerbExtractor())
    ])),
    ('clf', MultiOutputClassifier(clf))
])



In [34]:
param_grid = {
    'clf__estimator__n_estimators': [50, 100, 200, 500],
    'clf__estimator__learning_rate': [0.01, 0.1, 0.5, 1.0]
}

cv2 = GridSearchCV(model_2, param_grid= , cv=5, n_jobs=-1, verbose=3) 
# verbose=3 to get real time training progress
# n_jobs=-1 -> to train in parallel across the maximum number of cores in our computer, spending it up.
# cv=5 -> 5-fold cross validation


cv2.fit(X_train, y_train)
y_pred_test_2 = cv2.predict(X_test)

# classification report on test data
print(classification_report(y_test.values, y_pred_test_2, target_names=y.columns.values))

Fitting 5 folds for each of 12 candidates, totalling 60 fits


/Users/macos/Library/Python/3.9/lib/python/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/macos/Library/Python/3.9/lib/python/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/macos/Library/Python/3.9/lib/python/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/macos/Library/Python/3.9/lib/python/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/macos/Library/Python/3.9/lib/python/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is no

[CV 1/5] END clf__estimator__max_depth=10, clf__estimator__n_estimators=10;, score=0.201 total time= 1.9min
[CV 4/5] END clf__estimator__max_depth=10, clf__estimator__n_estimators=10;, score=0.197 total time= 1.9min


/Users/macos/Library/Python/3.9/lib/python/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/macos/Library/Python/3.9/lib/python/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 5/5] END clf__estimator__max_depth=10, clf__estimator__n_estimators=10;, score=0.192 total time= 1.9min


...

/Users/macos/Library/Python/3.9/lib/python/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 3/5] END clf__estimator__max_depth=40, clf__estimator__n_estimators=20;, score=0.199 total time= 2.3min
[CV 4/5] END clf__estimator__max_depth=40, clf__estimator__n_estimators=20;, score=0.199 total time= 2.3min
[CV 5/5] END clf__estimator__max_depth=40, clf__estimator__n_estimators=20;, score=0.190 total time= 2.3min
[CV 1/5] END clf__estimator__max_depth=40, clf__estimator__n_estimators=30;, score=0.201 total time= 2.6min
[CV 2/5] END clf__estimator__max_depth=40, clf__estimator__n_estimators=30;, score=0.188 total time= 2.4min
[CV 3/5] END clf__estimator__max_depth=40, clf__estimator__n_estimators=30;, score=0.195 total time= 2.3min
[CV 5/5] END clf__estimator__max_depth=40, clf__estimator__n_estimators=30;, score=0.190 total time= 2.1min
[CV 4/5] END clf__estimator__max_depth=40, clf__estimator__n_estimators=30;, score=0.190 total time= 2.1min


/Users/macos/Library/Python/3.9/lib/python/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


                        precision    recall  f1-score   support

               related       0.76      1.00      0.87      5008
               request       0.00      0.00      0.00      1098
                 offer       0.00      0.00      0.00        27
           aid_related       0.91      0.11      0.20      2712
          medical_help       0.00      0.00      0.00       532
      medical_products       0.00      0.00      0.00       346
     search_and_rescue       0.00      0.00      0.00       184
              security       0.00      0.00      0.00       126
              military       0.00      0.00      0.00       233
                 water       0.00      0.00      0.00       412
                  food       0.00      0.00      0.00       707
               shelter       0.00      0.00      0.00       590
              clothing       0.00      0.00      0.00       112
                 money       0.00      0.00      0.00       151
        missing_people       0.00      

/Users/macos/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/macos/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [38]:
cv2.best_params_

{'clf__estimator__max_depth': 10, 'clf__estimator__n_estimators': 30}

### 9. Export your model as a pickle file

In [40]:

# save model in pickle file
with open('classifier.pkl', 'wb') as f:
    pickle.dump(cv, f)

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.